In [1]:
pip install ipynb --upgrade

Requirement already up-to-date: ipynb in /Users/mattiaskallman1/anaconda3/lib/python3.7/site-packages (0.5.1)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pickle
import time
import os
import yaml
import sys
import pandas as pd

os.chdir("..")
import Models
os.chdir("MockPipeline")

os.chdir("../../pipeline")
import ipynb.fs.full.processing as processing
import ipynb.fs.full.features as features
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
os.chdir("../models-decision-system/MockPipeline")

In [3]:
os.getcwd()


'/Users/mattiaskallman1/Documents/Arcada/2020-2021/Decision_support_system/models-decision-system/MockPipeline'

In [4]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [5]:
settings = load_yaml('mock_settings.yaml')

In [6]:
settings['ensemble']['models'][0]

{'linreg': {'split': {'train': 0.8, 'test': 0.2}}}

In [7]:
dataframe = processing.create_dataframe(settings)

In [8]:
dataset = features.add(dataframe, settings['features'])

In [9]:
dataset

,Close,Momentum,SD,Label
Date_Timestamp,,,,
2010-02-01,1.39200,-0.04649,14.842308,1.39737
2010-02-02,1.39737,-0.03928,13.972612,1.38969
2010-02-03,1.38969,-0.05070,11.302307,1.37395
2010-02-04,1.37395,-0.05393,11.014429,1.36763
2010-02-05,1.36763,-0.04356,11.636518,1.36409
...,...,...,...,...
2019-12-25,1.10944,0.00301,54.215370,1.11012
2019-12-26,1.11012,0.00089,50.207241,1.11713
2019-12-27,1.11713,0.00388,49.232845,1.11813


In [10]:
data_linreg = features.split(dataset, 'linreg', settings['ensemble']['models'][0]['linreg'])

In [11]:
linreg = Models.train_model(data_linreg, 'linreg', settings['ensemble']['models'][0])

In [12]:
data_lstm = features.split(dataset, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [13]:
settings['ensemble']['models'][1]['lstm']['layers']

[{'lstm': {'value': 120, 'activation': 'relu'}},
 {'dropout': {'value': 0.15}},
 {'dense': {'value': 50, 'activation': 'relu'}},
 {'dense': {'value': 1}}]

In [14]:
lstm = Models.train_model(data_lstm, 'lstm', settings['ensemble']['models'][1]['lstm'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1861 samples, validate on 620 samples
Epoch 1/20
1861/1861 [==============================] - 3s 2ms/sample - loss: 0.0287 - val_loss: 0.0161
Epoch 2/20
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0052 - val_loss: 0.0107
Epoch 3/20
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0037 - val_loss: 0.0018
Epoch 4/20
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0029 - val_loss: 0.0019
Epoch 5/20
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0024 - val_loss: 0.0027
Epoch 6/20
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0020 - val_loss: 0.0019
Epoch 7/20
1861/1861 [==============================] - 2s 1ms/sample - loss: 0.0018 - val_loss: 0.0012
Epoch 8/20
1861/1861 [============================

In [36]:
data_lstm['train']['features']
data_lstm['train']['labels']

array([0.8011018 , 0.78347297, 0.74734305, ..., 0.08552737, 0.08247446,
       0.09218409])

In [32]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.python.keras import Input, Model

from tcn import TCN, tcn_full_summary

batch_size, timesteps, input_dim = None, 20, 1


def get_x_y(size=1000):
    import numpy as np
    pos_indices = np.random.choice(size, size=int(size // 2), replace=False)
    x_train = np.zeros(shape=(size, timesteps, 1))
    y_train = np.zeros(shape=(size, 1))
    x_train[pos_indices, 0] = 1.0
    y_train[pos_indices, 0] = 1.0
    return x_train, y_train


i = Input(batch_shape=(batch_size, timesteps, input_dim))

o = TCN(return_sequences=False)(i)  # The TCN layers are here.
o = Dropout(0.15)(o)
o = Dense(50)(o)
o = Dense(1)(o)

m = Model(inputs=[i], outputs=[o])
m.compile(optimizer='adam', loss='mse')

tcn_full_summary(m, expand_residual_blocks=False)

x, y = get_x_y()
m.fit(x, y, epochs=10, validation_split=0.2)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 20, 1)]           0         
_________________________________________________________________
residual_block_0 (ResidualBl [(None, 20, 64), (None, 2 8576      
_________________________________________________________________
residual_block_1 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_2 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_3 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_4 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_5 (ResidualBl [(None, 20, 64), (None, 2 1651

In [33]:
m.predict(x)

array([[0.99353063],
       [0.00176288],
       [0.00176288],
       [0.99353063],
       [0.00176288],
       [0.99353063],
       [0.00176288],
       [0.00176288],
       [0.99353063],
       [0.99353063],
       [0.00176288],
       [0.99353063],
       [0.00176288],
       [0.00176288],
       [0.00176288],
       [0.99353063],
       [0.99353063],
       [0.00176288],
       [0.99353063],
       [0.00176288],
       [0.99353063],
       [0.99353063],
       [0.00176288],
       [0.00176288],
       [0.00176288],
       [0.99353063],
       [0.99353063],
       [0.00176288],
       [0.00176288],
       [0.99353063],
       [0.00176288],
       [0.99353075],
       [0.00176288],
       [0.99353063],
       [0.00176288],
       [0.00176288],
       [0.00176288],
       [0.00176288],
       [0.99353063],
       [0.99353063],
       [0.99353063],
       [0.99353063],
       [0.00176288],
       [0.00176288],
       [0.00176288],
       [0.00176288],
       [0.99353063],
       [0.993